# Task3

Index_X = FSR_for_coord

Index_y = x_coord, y_coord

Data = Splited by Time

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_2023-07-19_04-38-35/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_2023-07-19_04-38-35

## Best metric (RMSE)

0.5864

In [1]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.LSTM'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    imputer = trial.suggest_categorical('imputer', ['sklearn.impute.SimpleImputer'])
    if imputer == 'sklearn.impute.SimpleImputer':
        trial.suggest_categorical('imputer_args/strategy', [
            'mean',
            'median',
        ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': 'FSR_for_coord',
        'index_y': ['x_coord', 'y_coord'],
        'data_loader': 'fsr_data.get_index_splited_by_time'
    }

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=100,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='rmse',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='rmse',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='rmse',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

[I 2023-07-19 04:38:35,093] A new study created in memory with name: optuna


In [3]:
results = tuner.fit()

2023-07-19 04:38:37,272	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
2023-07-19 04:38:38,883	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-07-19 04:38:38,941	INFO wandb.py:320 -- Already logged into W&B.


Trial name,date,done,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
FSR_Trainable_0028facd,2023-07-19_05-01-57,True,DESKTOP-0P789CI,2,0.348666,0.10033,172.26.215.93,430276,0.701186,27.9579,12.8793,27.9579,1689710517,2,0028facd
FSR_Trainable_01df0bfb,2023-07-19_05-39-36,True,DESKTOP-0P789CI,64,0.331538,0.0972941,172.26.215.93,441302,0.624855,84.3835,1.30111,84.3835,1689712776,64,01df0bfb
FSR_Trainable_037f32d8,2023-07-19_05-04-15,True,DESKTOP-0P789CI,8,0.348308,0.10295,172.26.215.93,430777,0.687046,83.2215,10.1992,83.2215,1689710655,8,037f32d8
FSR_Trainable_03820ab4,2023-07-19_05-39-07,True,DESKTOP-0P789CI,100,0.313437,0.088133,172.26.215.93,440907,0.601156,203.87,2.30116,203.87,1689712747,100,03820ab4
FSR_Trainable_0500b8ec,2023-07-19_05-45-38,True,DESKTOP-0P789CI,8,0.368832,0.103501,172.26.215.93,445486,0.668075,17.7862,2.08755,17.7862,1689713138,8,0500b8ec
FSR_Trainable_062fe3c1,2023-07-19_05-43-25,True,DESKTOP-0P789CI,32,0.361568,0.102922,172.26.215.93,443758,0.653524,41.7076,1.13045,41.7076,1689713005,32,062fe3c1
FSR_Trainable_103e2280,2023-07-19_05-30-48,True,DESKTOP-0P789CI,100,0.311801,0.0937261,172.26.215.93,436003,0.600161,879.705,8.22896,879.705,1689712248,100,103e2280
FSR_Trainable_14b20cd1,2023-07-19_05-40-33,True,DESKTOP-0P789CI,100,0.319448,0.0899343,172.26.215.93,441482,0.607324,128.815,1.22133,128.815,1689712833,100,14b20cd1
FSR_Trainable_150c9986,2023-07-19_05-22-57,True,DESKTOP-0P789CI,4,0.351306,0.102773,172.26.215.93,437198,0.698904,38.1842,9.66907,38.1842,1689711777,4,150c9986
FSR_Trainable_15c95f2b,2023-07-19_04-59-28,True,DESKTOP-0P789CI,100,0.34988,0.0975154,172.26.215.93,426556,0.64387,914.873,9.51794,914.873,1689710368,100,15c95f2b


(_WandbLoggingActor pid=422329) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=422329) wandb: wandb version 0.15.5 is available!  To upgrade, please run:
(_WandbLoggingActor pid=422329) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=422329) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=422329) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-07-19_04-38-35/FSR_Trainable_99ae87a3_1_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleIm_2023-07-19_04-38-38/wandb/run-20230719_043849-99ae87a3
(_WandbLoggingActor pid=422329) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=422329) wandb: Syncing run FSR_Trainable_99ae87a3
(_WandbLoggingActor pid=422329) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=422329) wandb: 🚀 View run at https://wandb.ai/se